In [54]:
import sys
import os

# Agregar la ruta del directorio scripts
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

### Extract and Transform

Se convirtio el dataset `credits.csv` a formato `parquet`.

En la carpeta `scripts/convert_csv_to_parquet.py` puedes ver el funcionamiento de la funcion.
- Una vez transformado se recomienda eliminar el archivo .csv 

In [55]:
from convert_csv_to_parquet import convertir_csv_a_parquet

convertir_csv_a_parquet(
    r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\credits.csv',
    r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\credits.parquet'
)

In [56]:
import pandas as pd

df = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\raw\credits.parquet')
df.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


### Desanidemos las columnas. `cast`

In [57]:
# Vemos que la columna esta en formato str. Pasemoslo a listas
df['cast'].apply(type).value_counts() 

cast
<class 'str'>    45476
Name: count, dtype: int64

La funcion que convierte tipos se encuentra en la carpeta `scripts/` 

Puedes mirar su algoritmo en esa carpeta.

In [58]:
from convert_str_to_list_dict import convert_list_dict

df['cast'] = df['cast'].apply(convert_list_dict)

In [59]:
df['cast'].apply(type).value_counts() 

cast
<class 'list'>    45476
Name: count, dtype: int64

In [60]:
from desanidar_columnas import desanidar_columna

# Aplicamos la funcion que automatiza procesos de la carpeta scripts/
cast_desanidado = desanidar_columna(df, 'cast', 'cast_desanidado')

cast_desanidado.head(4)


,cast_id,character,credit_id,gender,id,name,order,profile_path
0,0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg


In [61]:
# Agregar la columna 'id' del df original como 'id_df' a cast_desanidado. A travez de la union de cast_id.
cast_desanidado = cast_desanidado.merge(df[['cast_id', 'id']], on='cast_id', how='left', suffixes=('', '_df'))

In [62]:
# Vemos que son 564892 filas. Asi que trataremos de ver si contienen nulos que podamos eliminar.
# Muestra un porcentaje de nulos de cada columna.
avg_nulos = cast_desanidado.isnull().mean() * 100
avg_nulos


cast_id          0.000000
character        0.428046
credit_id        0.428046
gender           0.428046
id               0.428046
name             0.428046
order            0.428046
profile_path    31.204903
id_df            0.000000
dtype: float64

In [63]:
# Identificar las columnas con nulos
columnas_con_nulos = ['character', 'credit_id', 'gender', 'id', 'name', 'order']
# Identificar las filas nulas.
nulos = cast_desanidado[cast_desanidado[columnas_con_nulos].isnull().any(axis=1)]

nulos

,cast_id,character,credit_id,gender,id,name,order,profile_path,id_df
2274,137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124639
3971,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43475
6666,393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42981
7331,438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24257
10124,595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124472
...,...,...,...,...,...,...,...,...,...
564717,45447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455661
564759,45452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44330
564766,45458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122036
564781,45462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276895


In [64]:
# Obtener los índices de las filas con nulos
indices_nulos = nulos.index
# Eliminar nulos.
cast_desanidado = cast_desanidado.drop(index=indices_nulos)

### Desanidemos `crew`

In [65]:
# Vemos que la columna esta en formato str. Pasemoslo a listas
df['crew'].apply(type).value_counts() 

crew
<class 'str'>    45476
Name: count, dtype: int64

In [66]:
from convert_str_to_list_dict import convert_list_dict

df['crew'] = df['crew'].apply(convert_list_dict)

In [67]:
df['crew'].apply(type).value_counts() 

crew
<class 'list'>    45476
Name: count, dtype: int64

In [68]:
from desanidar_columnas import desanidar_columna

# Aplicamos la funcion que automatiza procesos de la carpeta scripts/
crew_desanidado = desanidar_columna(df, 'crew', 'crew_desanidado',1)
 
crew_desanidado

,credit_id,department,gender,id,job,name,profile_path,crew_id
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,1
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,1
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,1
3,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,1
4,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,1
...,...,...,...,...,...,...,...,...
465080,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None,45474
465081,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None,45474
465082,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg,45475
465083,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None,45475


In [69]:
# Agregar la columna 'id' del df original como 'id_df' a crew_desanidado. A travez de la union de crew_id.
crew_desanidado = crew_desanidado.merge(df[['crew_id', 'id']], on='crew_id', how='left', suffixes=('', '_df'))

In [70]:
# Vemos que son 465085 filas. Asi que trataremos de ver si contienen nulos que podamos eliminar.
# Muestra un porcentaje de nulos de cada columna.
avg_nulos = crew_desanidado.isnull().mean() * 100
avg_nulos

credit_id        0.165776
department       0.165776
gender           0.165776
id               0.165776
job              0.165776
name             0.165776
profile_path    79.552555
crew_id          0.000000
id_df            0.000000
dtype: float64

In [71]:
# Identificar las columnas con nulos
columnas_con_nulos = ['credit_id', 'department','gender', 'id', 'job', 'name']
# Identificar las filas nulas.
nulos = crew_desanidado[crew_desanidado[columnas_con_nulos].isnull().any(axis=1)]

nulos

,credit_id,department,gender,id,job,name,profile_path,crew_id,id_df
3486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190,56088
10351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615,123505
10527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,636,339428
10941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,662,318177
11387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,712,365371
...,...,...,...,...,...,...,...,...,...
463769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45237,332543
463954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45262,28469
464056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45278,458618
464444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45349,335251


In [72]:
# Obtener los índices de las filas con nulos
indices_nulos = nulos.index
# Eliminar nulos.
crew_desanidado = crew_desanidado.drop(index=indices_nulos)

Cambios necesarios para la API

In [83]:
# Cambiamos a minisculas las actores para que la api no diferencie entre minusculas y mayusculas.
cast_desanidado['name'] = cast_desanidado['name'].str.lower()
cast_desanidado['id'] = cast_desanidado['id'].astype(int)

In [100]:
filtro_actor = cast_desanidado[cast_desanidado['name'] == 'tom hanks']
cantidad_peliculas = filtro_actor['id_df'].nunique()
filtro_actor

,cast_id,character,credit_id,gender,id,name,order,profile_path,id_df
0,0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31,tom hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862
2394,147,Jim Lovell,52fe4253c3a36847f801595d,2.0,31,tom hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,568
5861,351,Forrest Gump,52fe420ec3a36847f800074f,2.0,31,tom hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,13
8493,503,Andrew Beckett,52fe452fc3a36847f80c1111,2.0,31,tom hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,9800
9087,534,Sam Baldwin,52fe4283c3a36847f8024bd9,2.0,31,tom hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,858
...,...,...,...,...,...,...,...,...,...
506903,39486,Chesley 'Sully' Sullenberger,561c1d9cc3a3682251002307,2.0,31,tom hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,363676
522062,40962,Robert Langdon,5475a40dc3a3687fd9000e10,2.0,31,tom hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,207932
540924,42897,Reader (voice),52fe4a909251416c750e6647,2.0,31,tom hanks,7.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,140465
544069,43183,Mr. Macauley,547200b79251411752000075,2.0,31,tom hanks,5.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,305642


Asi quedo nuestro dataframe con sus 2 tablas extras. 

- `cast_desanidado`
- `crew_desanidado`

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cast     45476 non-null  object
 1   crew     45476 non-null  object
 2   id       45476 non-null  int64 
 3   cast_id  45476 non-null  int64 
 4   crew_id  45476 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.7+ MB


# Load

Exportemos las tablas a la carpeta de processed
- Como ya hemos desanidado las listas. volveremos a pasarlas a str, para poder exportarlas.

In [74]:
from convert_list_to_str import lista_a_str

# Aplicar la función de conversión a la columna
df['cast'] = df['cast'].apply(lista_a_str)
df['crew'] = df['crew'].apply(lista_a_str)

In [75]:
# Asegúrate de que el directorio de destino exista
output_dir = '../data/processed/credits/'
os.makedirs(output_dir, exist_ok=True)

El df principal ya no nos servira, ya que tenemos las 2 tablas que conectan df_id con el id de movies_dataset.

Exportamos las tablas extras.

In [81]:
# Ruta y nombre del archivo a guardar.
parquet_path = os.path.join(output_dir, 'cast_desanidado.parquet')
# Guardar el DataFrame en la carpeta data/processed/movies
cast_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a 'data/processed/credits/'")

Datos exportados correctamente a 'data/processed/credits/'


In [77]:
# Ruta y nombre del archivo a guardar.
parquet_path = os.path.join(output_dir, 'crew_desanidado.parquet')
# Guardar el DataFrame en la carpeta data/processed/movies
crew_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a 'data/processed/credits/'")

Datos exportados correctamente a 'data/processed/credits/'
